# Optimization for Collabrative filtering

# A. Preprocessing of data

1. Data format

We are going to preprocess a rating file in the following csv format:  
```
UserID::MovieID::Rating::Timestamp
```

2. Prepare data for cross-validation

Splitting Data for a user-based N-fold cross-validation. Store each partition into a new csv file. 

It is convinient to use a python package lenskit. It can be installed by following https://lkpy.readthedocs.io/en/stable/install.html

Use the function lenskit.crossfold.partition_rows to partition all the ratings into N train-test partitions.

3. Convert to a list format

Convert data into a list format for fast processing


In [11]:
!pip install lenskit

In [14]:
import pandas as pd

# read a DataFrame of ratings from the csv ratings file
csvroot = 'data'
ratings = pd.read_csv(csvroot + '/ratings.csv')
ratings = ratings.rename(columns={'userId': 'user', 'movieId': 'item'})

def unique(list1):
    # insert the list to the set
    list_set = set(list1)
    # convert the set to the list
    unique_list = (list(list_set))
    # use a dictionary to store the index of each element in the list
    unique_dict = {}
    for i in range(len(unique_list)):
        unique_dict[unique_list[i]] = i 
    return unique_list, unique_dict

lst_users, dic_users = unique(ratings['user'])
lst_items, dic_items = unique(ratings['item'])

M = len(lst_users)
N = len(lst_items)
matrixSparsity = len(ratings) / (M*N)
print("We have %d users, %d movies and the rating matrix has %f percent of non-zero value.\n" % (M, N, 100*matrixSparsity))

We have 610 users, 9724 movies and the rating matrix has 1.699968 percent of non-zero value.



In [15]:
# 2 fold cross-validation, store each partition (both train and test) in a seperate csv file.
# Point 1: report the number of (user,iter,rating) items in each file.

import lenskit.crossfold as xf

for i, tp in enumerate(xf.partition_users(ratings, 5, xf.SampleN(5))):
    tp.train.to_csv(csvroot + '/train-%d.csv' % (i,))
    tp.test.to_csv(csvroot + '/test-%d.csv' % (i,))


In [ ]:
def convert_DF_to_RDD(DF):
    """ 
    This function converts a rating dataframe into a dictionary of lists
    Args:
        DF: a dataframe which contains 'user', 'item' and 'rating' columns
    Returns:
        RDD: a dictionary which contains 
            'total': the total number of rating
            'users': a list of user id for each rating
            'items': a list of item id for each rating
            'ratings': a list of ratings
    """ 
    RDD = {'total':0,'users':[],'items':[],'ratings':[]} 
    
    ### TODO

    
    return RDD

In [ ]:
# read the csv files from a partition of the cross-validation
# convert them to RDD using convert_DF_to_RDD

### TODO


# B. Gradient-descent algorithm

Based on the preprocessing, we are going to develop a method to find optimal P and Q on training data. It contains: 

1. compute the objective and the gradient of the objective function
2. implement the gradient-descent algroithm
3. measure the speed of this method

In [ ]:
# assume now you have obtained trainRDD and testRDD
# Compute the objective funtion

import numpy as np

def computeMSE(RDD,P,Q,la=0):
    """ 
    This function computes regularized Mean Squared Error (MSE)
    Args:
        RDD: a dict of list of userID, itemID, Rating
        P: user's features matrix (M by K)
        Q: item's features matrix (N by K)
        la: lambda parameter for regulization (see definition in the project description)
    Returns:
        mse: mean squared error
    """ 
    
    ### TODO
    

In [ ]:
# use a random P and Q to test the function computeMSE
# Point 2: report MSE with LAMBDA=0

K = 5 # rank parameter
P = np.random.rand(M,K) # user's features matrix (M by K)
Q = np.random.rand(N,K) # item's features matrix (N by K)

### TODO


In [ ]:
# Compute the gradient of the objective funtion
def computeGradMSE(RDD,P,Q,la=0):
    """ 
    This function computes the gradient of regularized MSE with respect to P and Q
    Args:
        RDD: a dict of list of userID, itemID, Rating
        P: user's features matrix (M by K)
        Q: item's features matrix (N by K)
    Returns:
        gradP, gradQ: gradient of mse with respect to each element of P and Q 
    """ 
    
    ### TODO
    

In [ ]:
# implement the (steepest) gradient-descent algorithm

### TODO


def GD(RDD,M,N,K,MAXITER=50, GAMMA=0.001, LAMBDA=0.05, adaptive=0):
    """ 
    This function implemnts the gradient-descent method to minimize the regularized MSE with respect to P and Q
    Args:
        RDD: a dict of list of users, items, ratings
        M: number of users
        N: number of items
        K: rank parameter
        MAXITER: maximal number of iterations (epoches) of GD 
        GAMMA: step size of GD
        LAMBDA: regulization parameter lambda in the mse loss
        adaptive: if 0 then use constant step size GD, 
                  if 1 then use line search to choose the step size automatically
    Returns:
        P: optimal P found by GD
        Q: optimal Q found by GD
        lreg_mse: a list of regulized mse values evaluated on RDD, after each iteration
        lmse: a list of mse values, evaluated on RDD after each iteration
        other scores for analysis purpose
    """ 
    
    ### TODO
    

In [ ]:
# Compare GD constant step size with GD line search step size
# Point 3: Make plots to show how (regularized) MSE changes with respect to GD iterations 
# Mention your initialization of P,Q, and the stopping criterion of GD

### TODO



# C. Perofomance evaluation
1. Compute RMSE score. You should use lenskit.metrics.predict.rmse for a fair comparison. Analyze both the training and test score on the 5 cross-validation partitions. 
2. Compare with a baseline method called Bias. Tune the hyper-parameters such as K and lambda to see if you can obtain a smaller RMSE. Try to explain why. 

In [ ]:
# Point 4: Report RMSE, together with your choice of K,LAMBDA and GD parameters

from lenskit.metrics.predict import rmse

def evaluteRMSE(RDD,P,Q):
    """ 
    This function computes the root MSE score on the rating of RDD. It compares the rating of each (i,j)
    in RDD, with the prediction made by <p_i,q_j>. 
    Args:
        RDD: a dict of list of users, items, ratings
        P: optimal P found by GD
        Q: optimal Q found by GD
    Returns:
        RMSE: the RMSE score
    """
    
    ### TODO
    

### TODO


In [ ]:
# Compare the performance with a baseline method called Bias
# see in https://lkpy.readthedocs.io/en/stable/bias.html
# Point 5:  report the RMSE of the baseline method, and analyze the results
# Hint: use read_csv in panda to read you csv data

from lenskit.algorithms.bias import Bias
from lenskit.batch import predict

### TODO
